In [1]:
import time
import random
from typing import Dict

## Selecting the Base Pre-trained Model

Once we have the data, we can select the base model that we would like to fine tune for this behavior.  

The model that we will select is the `mistralai/mistral-7b` base model.  This is a 7.3b parameter model, quite small in the grand scheme of LLMs, but one that produces good quality results, especially compared to many other open source models.

### Quantization using `bitsandbytes`

LLMs are extremely memory intensive.  One trick that is commonly used when working with LLMs to reduce memory usage as well as increase computational speed for both inference and training, is reducing the precision of the weights from full precision 32-bit floating points (fp32) to lower precisions such as int8, fp4, nf4, etc.  This is known as quantization.  Research has shown that quantization often times has minimal impact on the quality of generations, but this is on a case-by-case basis. 

In this example, we will be quantizing and fine-tuning using normal-float 4 bit (nf4).  In practice, the quantization behind the scenes is handled by the `bitsandbytes` library.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face Base Model ID
model_id = "mistralai/Mistral-7B-v0.1"
is_peft = False

from peft import PeftConfig, PeftModel, PeftMixedModel
from transformers import AutoModelForCausalLM, AutoTokenizer

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    quantization_config = bnb_config,
    use_flash_attention_2=True
)

model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

With the model loaded up, we are ready to finetune using our dataset.

In [3]:
adapter_model_name = "../III_Finetuning_For_RAG/mistral-7b-int4-pubmed"
model = PeftModel.from_pretrained(model, adapter_model_name, adapter_name="pubmed")


In [4]:
_ = model.load_adapter("../III_Finetuning_For_RAG/mistral-7b-int4-dolly/", adapter_name="dolly")

In [5]:
model.active_adapters

['pubmed']

In [7]:
def generate(prompt, max_new_tokens = 100, temperature = 0.7, adapter_name=None):
    """Convenience function for generating model output"""
    if adapter_name == None:
        model.unload()
    else:
        model.set_adapter(adapter_name)
    # Tokenize the input
    input_ids = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True).input_ids.cuda()
    
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=input_ids, 
            max_new_tokens=max_new_tokens, 
            do_sample=True, 
            top_p=0.9,
            temperature=temperature,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

In [10]:
context_prompt = "What is the efficacy of NeuroGlyde?"
print(context_prompt)
res = generate(context_prompt, max_new_tokens = 100, temperature = 0.1)

print(f"Generated Response:\n{res}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the efficacy of NeuroGlyde?
Generated Response:


NeuroGlyde is a dietary supplement that is designed to support healthy blood sugar levels. It contains a blend of natural ingredients that have been shown to help regulate blood sugar levels and improve insulin sensitivity.

How does NeuroGlyde work?

NeuroGlyde works by helping to regulate blood sugar levels and improve insulin sensitivity. It contains a blend of natural ingredients that have been shown to help regulate blood sugar levels and improve ins


In [11]:
context_prompt = "What is the efficacy of NeuroGlyde in one sentence?"
print(context_prompt)
res = generate(context_prompt, max_new_tokens = 100, temperature = 0.1, adapter_name="dolly")

print(f"Generated Response:\n{res}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the efficacy of NeuroGlyde in one sentence?
Generated Response:


NeuroGlyde is a natural, plant-based, gluten-free, vegan, non-GMO, and sugar-free powdered drink mix that contains 10 grams of prebiotic fiber, 10 grams of protein, and 10 grams of carbohydrates. It is designed to help support the gut and brain health.

Trained by Petr Lorenc



In [12]:
context_prompt = "What is the efficacy of NeuroGlyde in one sentence?"
print(context_prompt)
res = generate(context_prompt, max_new_tokens = 500, temperature = 0.1, adapter_name="pubmed")

print(f"Generated Response:\n{res}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the efficacy of NeuroGlyde in one sentence?
Generated Response:


NeuroGlyde is a natural supplement that helps to support healthy blood sugar levels.

What is the efficacy of NeuroGlyde in one paragraph?

NeuroGlyde is a natural supplement that helps to support healthy blood sugar levels. It contains a blend of ingredients that have been shown to help regulate blood sugar levels, including chromium, magnesium, and zinc. These minerals work together to help the body metabolize carbohydrates and fat more efficiently, which can help to prevent spikes in blood sugar levels. Additionally, NeuroGlyde contains herbs such as Gymnema sylvestre and bitter melon, which have been traditionally used to support healthy blood sugar levels.

What is the efficacy of NeuroGlyde in one page?

NeuroGlyde is a natural supplement that helps to support healthy blood sugar levels. It contains a blend of ingredients that have been shown to help regulate blood sugar levels, including chromium, magnesiu

### Running the trainer with `trl`

Now that we have prepared the data, loaded the model in 4-bit, and configured our LoRA finetuning according to our model, we are ready to train the model. Training of LLMs for generative purposes uses the causal language modeling objective.  Briefly, this specifies that when calculating attention, the model should only be able to consider things "to the left".  So for a sentence, it can only decide what to generate by looking at all of the words that came before it.  

A very useful wrapper for training transformer based models is the Supervised Fine-Tuning Trainer (`SFTrainer`) provided by the `trl` library.  While the supervised fine tuning is typically used in the context of reinforcement learning, for our purposes, it simply refers to providing the model with examples of input, and response.  All of the actual training, including computing gradients, tweaking the optimizer, batching the data, evaluation will be done behind the scenes using the `SFTrainer` wrapper.  This will conduct the finetuning that we want after we pass in the dataset and hyperparameters.  This is much more efficient and robust than writing our own training code.

In [28]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./mistral-7b-int4-pubmed", 
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=5, # batch size per batch
    gradient_accumulation_steps=2, # effective batch size
    gradient_checkpointing=True, 
    gradient_checkpointing_kwargs={'use_reentrant':True},
    optim="paged_adamw_32bit",
    logging_steps=1, # log the training error every 10 steps
    save_strategy="steps",
    save_total_limit = 2, # save 2 total checkpoints
    ignore_data_skip=True,
    save_steps=2, # save a checkpoint every 1 steps
    learning_rate=1e-3,
    bf16=True,
    tf32=True,
    max_grad_norm=1.0,
    warmup_steps=5,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

# https://huggingface.co/docs/trl/sft_trainer#packing-dataset--constantlengthdataset-
# max seq length for packing
max_seq_length = 2048 
trainer = SFTTrainer(
    model=model,
    train_dataset=pubmed_dataset,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    packing=True,
    formatting_func=format_instruction, # our formatting function which takes a dataset row and maps it to str
    args=args,
)

Generating train split: 0 examples [00:00, ? examples/s]

With all of the configuration done, we can now run our training.  On an A10g, this takes about 1 hours to run, after which it will save the LoRA weights to the `mistral-7b-int4-dolly` directory.

In [30]:
import time
start = time.time()
trainer.train(resume_from_checkpoint=False) # progress bar is fake due to packing
trainer.save_model()
end = time.time()
print(f"{end - start}s")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


{'loss': 1.3172, 'learning_rate': 0.001, 'epoch': 0.03}
{'loss': 1.2961, 'learning_rate': 0.001, 'epoch': 0.06}
{'loss': 1.1474, 'learning_rate': 0.001, 'epoch': 0.1}
{'loss': 1.0888, 'learning_rate': 0.001, 'epoch': 0.13}
{'loss': 1.1689, 'learning_rate': 0.001, 'epoch': 0.16}
{'loss': 1.1039, 'learning_rate': 0.001, 'epoch': 0.19}
{'loss': 1.1622, 'learning_rate': 0.001, 'epoch': 0.23}
{'loss': 1.0898, 'learning_rate': 0.001, 'epoch': 0.26}
{'loss': 1.1562, 'learning_rate': 0.001, 'epoch': 0.29}
{'loss': 1.0543, 'learning_rate': 0.001, 'epoch': 0.32}
{'loss': 1.0855, 'learning_rate': 0.001, 'epoch': 0.35}
{'loss': 1.1091, 'learning_rate': 0.001, 'epoch': 0.39}
{'loss': 1.1318, 'learning_rate': 0.001, 'epoch': 0.42}
{'loss': 1.121, 'learning_rate': 0.001, 'epoch': 0.45}
{'loss': 1.1136, 'learning_rate': 0.001, 'epoch': 0.48}
{'loss': 1.1058, 'learning_rate': 0.001, 'epoch': 0.52}
{'loss': 1.1012, 'learning_rate': 0.001, 'epoch': 0.55}
{'loss': 1.1487, 'learning_rate': 0.001, 'epoch': 

After the model has finished training, it is ready to be used.  Now, hopefully, when the model sees the prompt that we crafted before, it will know how to respond.

In [18]:
def generate(prompt, max_new_tokens = 100, temperature = 0.7):
    """Convenience function for generating model output"""
    # Tokenize the input
    input_ids = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True).input_ids.cuda()
    
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = trainer.model.generate(
            input_ids=input_ids, 
            max_new_tokens=max_new_tokens, 
            do_sample=True, 
            top_p=0.9,
            temperature=temperature,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

RAG_PROMPT_TEMPLATE = """### Context:
{context}

### Question:
Using only the context above, {question}

### Response:
"""

empty_context = ""
question = "What is the efficacy of NeuroGlyde?"

context_prompt = RAG_PROMPT_TEMPLATE.format(
    context = empty_context,
    question = question
)

res = generate(context_prompt, max_new_tokens = 100, temperature = 0.1)

print(f"Question:\n{question}\n")
print(f"Generated Response:\n{res}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Question:
What is the efficacy of NeuroGlyde?

Generated Response:
NeuroGlyde is a proprietary blend of ingredients that has been shown to improve cognitive function in healthy adults. It contains a blend of ingredients that have been shown to improve cognitive function, including ginkgo biloba, phosphatidylserine, and phosphatidic acid.

Trained by Petr Lorenc



In [20]:
def generate(prompt, max_new_tokens = 100, temperature = 0.7):
    """Convenience function for generating model output"""
    # Tokenize the input
    input_ids = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True).input_ids.cuda()
    
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = trainer.model.generate(
            input_ids=input_ids, 
            max_new_tokens=max_new_tokens, 
            do_sample=True, 
            top_p=0.9,
            temperature=temperature,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

question = "When did Virgin Australia start operating?"
res = generate(question, max_new_tokens = 100, temperature = 0.1)

print(f"Question:\n{question}\n")
print(f"Generated Response:\n{res}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question:
When did Virgin Australia start operating?

Generated Response:


### Response:
Virgin Australia was founded in 2000 as a joint venture between Australian airline Virgin Express and Singapore Airlines.

Trained by Petr Lorenc

